In [4]:
import pandas as pd
import altair as alt
from functools import reduce
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
# data selection
df_raw = pd.read_csv('data.csv')
df = df_raw.copy()

In [6]:
def sample(df, n):
    df = pd.concat([
        df[df['Diabetes_012'] == 0].sample(n // 3),
        df[df['Diabetes_012'] == 1].sample(n // 3),
        df[df['Diabetes_012'] == 2].sample(n // 3)
    ])

    df = df.sample(frac=1).reset_index(drop=True)

    return df

In [7]:
df_1000 = sample(df, 1000)

df_raw = df_raw[
    ['Smoker', 'Fruits', 'Veggies', 'Age', 'PhysHlth', 'MentHlth',
     'Income', 'Education', 'Diabetes_012']
]

df_10000, _ = train_test_split(
    df_raw,
    stratify=df_raw['Diabetes_012'],
    train_size=1000,
)

In [8]:
# data translation
color_dict = {0: 'blue', 1: 'orange', 2: 'red'}
diabetes = {0: 'No Diabetes', 1: 'Pre-Diabetes', 2: 'Diabetes'}

education = {
    x + 1 : 'High School or Less' for x in range(4)
}
education[5] = 'Some College'
education[6] = 'Bachelor\'s or Higher'

age_brackets = {x: f'{15 + 5 * x}-{19 + 5 * x}' for x in range(2, 13)}
age_brackets[1] = '18-24'
age_brackets[13] = '80+'

binary = {0: 'No', 1: 'Yes'}

health_bins = [0, 5, 10, 31]
health_labels = ['High', 'Medium', 'Low']

income_bins = [0, 4, 7, 8.1]

df_10000['AgeBkt'] = df_10000['Age'].apply(lambda x: age_brackets[int(x)])
df_10000['Diabetes'] = df_10000['Diabetes_012'].apply(lambda x: diabetes[x])
df_10000['Education Level'] = df_10000['Education'].apply(lambda x: education[x])
df_10000['Fruits and Veggies'] = df_10000['Fruits'].astype(int) & df_10000['Veggies'].astype(int)
df_10000['Physical Health Level'] = pd.cut(df_10000['PhysHlth'], bins=health_bins, labels=health_labels, right=False)
df_10000['Mental Health Level'] = pd.cut(df_10000['MentHlth'], bins=health_bins, labels=health_labels, right=False)
df_10000['Income Level'] = pd.cut(df_10000['Income'], bins=income_bins, labels=health_labels, right=False)

for col in ['Fruits and Veggies', 'Smoker']:
    df_10000[col] = df_10000[col].apply(lambda x: binary[x])

In [9]:
controllable = ['Smoker', 'Physical Health Level', 'Fruits and Veggies',
                'Income Level', 'Mental Health Level', 'Education Level']
output = ['Diabetes']

In [10]:
# create selection widgets
dropdowns = [
    alt.binding_select(options=
                       df_10000[feature].unique().tolist(),
                       name=f'{feature}: '
                       )
    for feature in controllable
]

selects = [
    alt.selection_point(
        fields=[feature], bind=dropdowns[i], value=df_10000[feature].unique()[0]
    )
    for i, feature in enumerate(controllable)
]

# combine all selections
combined_filter = reduce(lambda x, y: x & y, selects)

# create chart
chart = alt.Chart(df_10000).transform_filter(
    combined_filter
).transform_aggregate(
    count='count()', groupby=['AgeBkt', 'Diabetes']
).transform_joinaggregate(
    total='sum(count)', groupby=['AgeBkt']
).transform_calculate(
    Proportion='datum.count / datum.total'
).mark_bar().encode(
    x=alt.X('AgeBkt:O', title='Age Bucket'),
    y=alt.Y('Proportion:Q', stack='normalize', title='Proportion of Diabetes Diagnosis'),
    color=alt.Color(
        'Diabetes:N', scale=alt.Scale(domain=diabetes.values(), range=color_dict.values()),
        title='Diabetes Classification'),
    tooltip=['AgeBkt:O', 'Diabetes:N', 'Proportion:Q']
).add_params(
    *selects
).properties(
    width=600,
    height=400,
    title='Proportion of Diabetes Levels by Age Bucket'
)

chart.display()

alt.Chart(...)

In [11]:
def create_matrix(df, output_column='Diabetes_012'):
    features = df.drop(columns=[output_column])
    labels = df[output_column].reset_index(drop=True)

    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    similarity_matrix = cosine_similarity(features)
    np.fill_diagonal(similarity_matrix, -1)

    return similarity_matrix, labels

In [12]:
legend_handles = [
    mpatches.Patch(color='blue', label='No Diabetes'),
    mpatches.Patch(color='orange', label='Pre-Diabetes'),
    mpatches.Patch(color='red', label='Diabetes')
]

In [13]:
def create_network(matrix, labels, n):
    G = nx.Graph()

    for i, status in enumerate(labels):
        G.add_node(i, diabetes=status)

    for i in range(matrix.shape[0]):
        most_similar_idx = np.argpartition(matrix[i], -n)[-n:]

        for index in most_similar_idx:
            G.add_edge(i, index, weight=matrix[i, index])

    plt.figure(figsize=(7, 7), dpi=1000)
    pos = None #nx.spring_layout(G)

    colors = [color_dict[int(G.nodes[n]['diabetes'])] for n in G.nodes]

    nx.draw(
        G, pos,
        node_color=colors,
        with_labels=False,
        node_size=5,
        edge_color='grey',
        alpha=0.7
    )

    plt.legend(handles=legend_handles, loc='upper right')

In [14]:
strat_mat, strat_labels = create_matrix(df_1000)
create_network(strat_mat, strat_labels, n=1)
plt.title('Patient Similarity Network (Equal Sample) (n=1000)')
plt.show()